## Quandl API data wrangling project

This project pulls data from the Quandl API and answers questions on equities data from the Frankfurt Stock Exhange (FSE). Specifically, I focus on the stock prices of Carl Zeiss Meditec, a manufacturer of tools for eye examinations and medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock symbol AFX_X.

API key after registering on Quandl

In [98]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '123456'

In [99]:
import requests
import json

In [100]:
# Call Quandl API and pull a small sample of the data (JSON)

r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2019-04-01&end_date=2019-04-01&api_key='+API_KEY)

In [101]:
# Look at data structure
print(r.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2019-04-01', 'end_date': '2019-04-01', 'frequency': 'daily', 'data': [['2019-04-01', None, 76.0, 74.1, 74.9, None, 186845.0, 14041454.0, None, None, None]], 'collapse': None, 'order': None}}


&nbsp;

Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017.

In [102]:
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY)

&nbsp;

Convert the JSON object into a Python dictionary.

In [103]:
data = r.json()

In [104]:
data['dataset_data']['data'][0:2]

[['2017-12-29',
  51.76,
  51.94,
  51.45,
  51.76,
  None,
  34640.0,
  1792304.0,
  None,
  None,
  None],
 ['2017-12-28',
  51.65,
  51.82,
  51.43,
  51.6,
  None,
  40660.0,
  2099024.0,
  None,
  None,
  None]]

&nbsp;

Calculate what the highest and lowest opening prices were for this stock in this period.

In [105]:
# List of opening prices
opening_prices = [x[1] for x in data['dataset_data']['data'] if x[1] is not None]

# Find max price and date that the price occurred
maximum = max(opening_prices)
max_idx = opening_prices.index(maximum)
max_date = data['dataset_data']['data'][max_idx][0]

# Find min price and date that the price occurred
minimum = min(opening_prices)
min_idx = opening_prices.index(minimum)
min_date = data['dataset_data']['data'][min_idx][0]

print('Highest opening price was {} on {}'.format(maximum, max_date))
print('Lowest opening price was {} on {}'.format(minimum, min_date))

Highest opening price was 53.11 on 2017-12-14
Lowest opening price was 34.0 on 2017-01-27


&nbsp;

What was the largest change in any one day (based on High and Low price)?

In [106]:
# List of high prices
high_prices = [x[2] for x in data['dataset_data']['data'] if x[2] is not None]

# List of low prices
low_prices = [x[3] for x in data['dataset_data']['data'] if x[3] is not None]

# Difference between high and low in a given day
diff = [hi-lo for hi, lo in zip(high_prices, low_prices)]

print('Largest change in any one day: {0:.2f}'.format(max(diff)))

Largest change in any one day: 2.81


&nbsp;

What was the largest change between any two days (based on Closing Price)?

In [107]:
# Difference in closing price between two consecutive days
day1 = [x[4] for x in data['dataset_data']['data'][:-1]]
day2 = [x[4] for x in data['dataset_data']['data'][1:]]
diff_closing = [abs(y-x) for x, y in zip(day1, day2)]
max_diff = max(diff_closing)

print('Largest change between any two consecutive trading days: {0:.2f}'.format(max_diff))

Largest change between any two consecutive trading days: 2.56


In [108]:
# Difference in closing price between any two days of the year
closing_prices = [x[4] for x in data['dataset_data']['data']]
diff = max(closing_prices)-min(closing_prices)

print('Largest difference in closing price between any two days: {0:.2f}'.format(diff))

Largest difference in closing price between any two days: 19.03


&nbsp;

What was the average daily trading volume during this year?

In [109]:
daily_traded_vol = [x[6] for x in data['dataset_data']['data']]
avg_daily_traded_vol = sum(daily_traded_vol)/len(daily_traded_vol)
print('Average daily trading volume in 2017: {0:.2f}'.format(avg_daily_traded_vol))

Average daily trading volume in 2017: 89124.34


&nbsp;

What was the median trading volume during this year?

In [110]:
def median(lst):
    """This function takes a list of values and returns the median value."""
    # sort the values in order
    lst.sort()
    
    # find the center index to access in the list
    idx = round(len(lst)/2)-1
    
    return lst[idx]

In [111]:
print('Median trading volume in 2017: {0:.2f}'.format(median(daily_traded_vol)))

Median trading volume in 2017: 76286.00
